In [524]:
import requests
import urllib
import re
from bs4 import BeautifulSoup
from googlesearch import search
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
import time
import pickle

TownCharts.com will provide the initial set of variables.
Other variables might be merged after ...

First collect relevant variables from each figures that display consistent information from each of the tabs from San Luis Obispo: Demographics, Housing,Economy,Education,Healthcare. 

And then automate since its a structured website for cities from major metropolitan areas. 

First up is the housing the variables. Out of the 44 figures 18(3,9,13,16,19,20,22,23,25,26,27,31,32,33,34,35,38,39) will not be included due to incosistencies. Some cities might not have the figure information and an NA will be put in place. 

Note figure 24:will have several variables


In [550]:
url = 'https://www.towncharts.com/California/Housing/San-Luis-Obispo-city-CA-Housing-data.html'
r = requests.get('https://www.towncharts.com/California/Housing/San-Luis-Obispo-city-CA-Housing-data.html')
soup = BeautifulSoup(r.text,'html.parser')

In [551]:
if r.status_code !=200:
    print('yo')



# **Note:Each housing tab has 3 vital sections in total**

In [552]:
def get_text_section(target_section):
    found = soup.find_all('div',attrs = {'class':target_section})
    validsection = found[0]
    text_section = validsection.find_all(text=True)
    return text_section
housing_text_section1 = get_text_section('section1')
housing_text_section2 = get_text_section('section2')
housing_text_section3 = get_text_section('section4')

Data dictionary:
<br>
    housing_units: total number of housing units for the city
    <br>
    housing_density: housing density in the area as measured     by houses per square land mile
    change_hunits: the change in housing units
    

In [652]:
def refinestring(desiredreplacement,yourstring):
    """
    yourstring: a string which you extract only digits and decimals
    desiredreplacement: a dictonary definining your replacement values
    
    returns: your string with the desired replacements you request 
    """
    rep = dict((re.escape(k),v) for k,v in desiredreplacement.items())
    pattern =re.compile("|".join(rep.keys()))
    
    return float(pattern.sub(lambda m:rep[re.escape(m.group(0))],yourstring))

In [651]:
# rep = {"$":"",",":"","%":""}
# rep = dict((re.escape(k),v)for k,v in rep.items())
# pattern = re.compile("|".join(rep.keys()))
# test =float(pattern.sub(lambda m:rep[re.escape(m.group(0))],hcost_as_perc_of_hincome_ownmortg))

re.compile(r'\$|,|%', re.UNICODE)

In [619]:
def searchpattern(pattern,resultset):
    '''
    pattern:regex pattern
    resultset: A beautiful soup result set
    output housing variable of interest type string
    '''
    result = [i for i in resultset if re.search(pattern,i)]
    if result == []:
        result  = None
    elif result !=[]:
        result = (re.findall(pattern,result[0]))[0]
    return result

city_name = re.search('(.*)\sCalifornia',housing_text_section1[0]).group(1)

#for variables need to remove everything but numbers
#figure1 
housing_units = searchpattern('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_text_section1)
#extract only the digits no commas
housing_units = re.findall('\d+',housing_units)
housing_units =int("".join(housing_units))

#figure2 
housing_density = searchpattern('it\shas\s(\d.*?\d)\shousing\sdensity\s',housing_text_section1)
housing_density = float(housing_density.replace(',',""))

#figure4 
change_hunits = searchpattern('it\shas\s(.*?%)\schange\sin\s',housing_text_section1)
change_hunits = deletepercent(change_hunits)

#figure5
percent_of_rent_to_total = searchpattern('it\shas\s(\d.*?%)\sRenter\sPercent\s',housing_text_section1)
percent_of_rent_to_total = deletepercent(percent_of_rent_to_total)

#figure 6
owned_homes = searchpattern('it\shas\s(\d.*?%)\sOwner\sPercent\s',housing_text_section1)
owned_homes = deletepercent(owned_homes)

#figure 7 median home value (unique character  = $-numbers-commas)
med_homeval = searchpattern('it\shas\s(.*?\d)\smedian\shome\svalue\s',housing_text_section2)
med_homeval = 
#med_homeval = med_homeval.replace(',',"")

#figure 8 median rental rate
med_rental_rate = searchpattern('it\shas\s(.*?\d)\srental\srates\s',housing_text_section2)
#figure 10 median owner costs
med_owner_cost = searchpattern("it\shas\s(.*?\d)\smedian\sownership\scost\s",housing_text_section2)
#figure 11
med_OwnCostas_Perc_of_Income = searchpattern('median\sowner\scost\sas\sa\spercent\sof\stotal\shousehold\sincome\sand\sit\shas\s(\d.*?%)',housing_text_section2)

#figure 12 median home value as a percent of a median workers earnings
med_HCasPerc_of_medearn = searchpattern("it\shas\s(\d.*?%)\smedian\shome\svalue\sas\s",housing_text_section2)
#figure14
med_hcost_ownmortg=searchpattern('depicts.*\$(.*?\d)\smedian',housing_text_section2)
#figure 15
med_hcost_own_wo_mortg = searchpattern('are.*\$(.*?\d)\smedian\shousing\s',housing_text_section2)
#figure 17
hcost_as_perc_of_hincome_ownmortg= searchpattern('with\smortgages.*\s(\d.*?%)\scost',housing_text_section2)
#figure18
hcost_as_perc_of_hincome_womortg= searchpattern('This\sanalysis.*\s(\d.*?%)\scost',housing_text_section2)
#figure 21 **
med_real_estate_taxes =searchpattern('it\shas\s(.*?\d)\smedian\sreal\sestate\s',housing_text_section2)
#figure 24 biggie see below

#figure 28 format 3.0
family_members_per_hunit = searchpattern('it\shas\s(.*?\d)\sfamily\smembers\sper\s',housing_text_section3)
#figure 29 4.5
median_numof_rooms = searchpattern('it\shas\s(.*?\d)\saverage\snumber\sof\s',housing_text_section3)
#figure 30
median_year_house_built = searchpattern('it\shas\s(.*?\d)\syear\sbuilt\swhich\s',housing_text_section3)
#figure 36
household_size_of_howners = searchpattern('it\shas\s(.*?\d)\shomeowner\shousehold\ssize\s',housing_text_section3)
#figure 37
household_size_for_renters = searchpattern('it\shas\s(.*?\d)\srenter\shousehold\ssize\s',housing_text_section3)
#figure 40
med_year_moved_in_for_owners = searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\shome',housing_text_section3)
#figure41
med_year_renter_moved_in = searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\srenter',housing_text_section3)

TypeError: replace() argument 1 must be str, not list

In [648]:
rep = {"$":"",",":"","%":""}
rep = dict((re.escape(k),v)for k,v in rep.items())
pattern = re.compile("|".join(rep.keys()))
test =float(pattern.sub(lambda m:rep[re.escape(m.group(0))],hcost_as_perc_of_hincome_ownmortg))

In [653]:
test = refinestring({"$":"",",":"","%":""},med_owner_cost)
test

1523.0

In [647]:
hcost_as_perc_of_hincome_ownmortg

'26%'

In [644]:
med_hcost_own_wo_mortg

'450'

In [638]:
med_hcost_ownmortg

'2,279'

In [634]:
med_HCasPerc_of_medearn

'1155%'

In [631]:
med_OwnCostas_Perc_of_Income

'20%'

In [628]:
med_owner_cost

'$1,523'

In [625]:
med_rental_rate

'$1,403'

In [624]:
med_homeval

'$590,800'

In [ ]:
pattern.sub(lambda m:rep[re.escape(m.group(0))],med_homeval)

In [623]:
test

'590800'

In [618]:
med_homeval

'$590,800'

In [114]:
javascript = soup.find_all('script',attrs = {'type':'text/javascript'})
need = javascript[4]
needtext = need.find_all(text=True)
needtext[0]
needtext = str(needtext)

In [156]:
fig24 = re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext).group(1)
fig24

"[[\\'Rental Rates\\',\\'Studio-No Bedroom\\',\\'One Bedroom\\',\\'Two Bedroom\\',\\'Three Bedroom\\'],\\n\\t\\t[\\'$1,000 to $1,499\\',{v:0.249,f:\\'24.9%\\'},{v:0.391,f:\\'39.1%\\'},{v:0.349,f:\\'34.9%\\'},{v:0.18,f:\\'18.0%\\'}],\\n\\t\\t[\\'$1,500 or more\\',{v:0.083,f:\\'8.3%\\'},{v:0.176,f:\\'17.6%\\'},{v:0.495,f:\\'49.5%\\'},{v:0.643,f:\\'64.3%\\'}],\\n\\t\\t[\\'$300 to $499\\',{v:0.02,f:\\'2.0%\\'},{v:0.029,f:\\'2.9%\\'},{v:0.008,f:\\'0.8%\\'},{v:0.006,f:\\'0.6%\\'}],\\n\\t\\t[\\'$500 to $749\\',{v:0.082,f:\\'8.2%\\'},{v:0.065,f:\\'6.5%\\'},{v:0.046,f:\\'4.6%\\'},{v:0.073,f:\\'7.3%\\'}],\\n\\t\\t[\\'$750 to $999\\',{v:0.526,f:\\'52.6%\\'},{v:0.283,f:\\'28.3%\\'},{v:0.064,f:\\'6.4%\\'},{v:0.05,f:\\'5.0%\\'}],\\n\\t\\t[\\'Less than $300\\',{v:0.016,f:\\'1.6%\\'},{v:0.042,f:\\'4.2%\\'},{v:0.018,f:\\'1.8%\\'},{v:0.022,f:\\'2.2%\\'}],\\n\\t\\t[\\'No cash rent\\',{v:0.023,f:\\'2.3%\\'},{v:0.012,f:\\'1.2%\\'},{v:0.02,f:\\'2.0%\\'},{v:0.026,f:\\'2.6%\\'}]\\n\\t\\t]"

In [173]:
mrr = re.findall("v\:(.*?\d),",fig24)

In [178]:
studio_1000_1499 = mrr[0]
studio_1500_more= mrr[4]
studio_750_999 = mrr[16]

In [182]:
onebed_1000_1499 = mrr[1]
onebed_1500_more = mrr[5]
onebed_750_999 = mrr[17]

In [183]:
twobed_1000_1499 = mrr[2]
twobed_1500_more = mrr[6]
twobed_750_999 = mrr[18]

'0.283'

In [ ]:
threebed_1000_1499 = mrr[3]
threebed_1500_more = mrr[7]
threebed_750_999 = mrr[19]

In [11]:
print(family_members_per_hunit)

['3.0']


In [22]:
searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\shome\s',housing_text_section3)

['2002']

In [56]:
housing_text_section3

['San Luis Obispo city-CA California Home Cost Charts',
 'Looking at the housing data in ',
 'Figure 7',
 ' we start with home values for the area. ',
 'San Luis Obispo city-CA',
 ' shows it has $590,800 median home value which is the 6th in median home value out of 10 total in the area. The city with the highest median home value in the area is ',
 'Los Ranchos',
 ' with a value of $876,000 measures 48.3% larger. ',
 'Figure 8',
 ' shows the median rental rate in the area. San Luis Obispo city-CA shows it has $1,403 rental rates which is the 5th in rental rates out of 10 total in the area. The city with the highest rental rates in the area is ',
 'Santa Margarita',
 ' with a rate of $1,575 measures 12.3% larger. Comparing rental rates to the ',
 'United States',
 ' average of $982, San Luis Obispo city-CA measures 42.9% larger. Also, measured against the state of ',
 'California',
 ', rental rates of $1,358, San Luis Obispo city-CA is only about 3.3% larger. ',
 'Figure 9',
 ' is a fr

In [102]:
testing = [i for i in housing_text_section3 if re.search('it\shas\s(.*?\d)\sfamily\smembers\sper\s',i)]
print(testing)

[' shows the household size for families living in the area and it has 3.0 family members per housing unit which is the 2nd most of all the places in the area. The city with the highest family members per housing unit in the area is ']


In [121]:
# housing_units=[i for i in housing_text_section1 if re.search('it\shas\s(\d.*?\d)\shousing\sunits\s',i)]
# if housing_units == []:
#     housing_units = None
# else:
#     housing_units = re.findall('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_units[0])
    
    
        
# print(housing_units)
# housing_change = [i for i in housing_text_section1 if re.search('it\shas\s(.*?%)\schange\sin\s',i)]
# h_change = re.findall('it\shas\s(.*?%)\schange\sin\s',housing_change[0])
# print(h_change)
# Percent_of_Rent_to_Total = [i for i in housing_text_section1 if re.search('it\shas\s(\d.*?%)\sRenter\sPercent\s',i)]
# print(Percent_of_Rent_to_Total)

['20,332']


#Automation
#list of cities gathered from the major metropolitan areas

In [192]:
cities_data = pd.read_csv("C:\\Users\\Crist\\AppData\\Local\\Temp\\American-Community-Survey-Project\\cal_cities_lat_long.csv")

In [210]:
city_list = cities_data.Name.tolist()
list_of_cities = []
for city in city_list:
    new = city.replace(' ','-')
    list_of_cities.append(new)

In [316]:
list_of_cities

['Adelanto',
 'Agoura-Hills',
 'Alameda',
 'Albany',
 'Alhambra',
 'Aliso-Viejo',
 'Alturas',
 'Amador-City',
 'American-Canyon',
 'Anaheim',
 'Anderson',
 'Angels',
 'Antioch',
 'Arcadia',
 'Arcata',
 'Arroyo-Grande',
 'Artesia',
 'Arvin',
 'Atascadero',
 'Atwater',
 'Auburn',
 'Avalon',
 'Avenal',
 'Azusa',
 'Bakersfield',
 'Baldwin-Park',
 'Banning',
 'Barstow',
 'Beaumont',
 'Bell',
 'Bell-Gardens',
 'Bellflower',
 'Belmont',
 'Belvedere',
 'Berkeley',
 'Beverly-Hills',
 'Big-Bear-Lake',
 'Biggs',
 'Bishop',
 'Blue-Lake',
 'Blythe',
 'Bradbury',
 'Brawley',
 'Brea',
 'Brentwood',
 'Brisbane',
 'Buellton',
 'Buena-Park',
 'Burbank',
 'Burlingame',
 'Calabasas',
 'Calexico',
 'California-City',
 'Calimesa',
 'Calipatria',
 'Calistoga',
 'Camarillo',
 'Campbell',
 'Canyon-Lake',
 'Capitola',
 'Carlsbad',
 'Carmel-By-the-Sea',
 'Carpinteria',
 'Carson',
 'Cathedral-City',
 'Ceres',
 'Cerritos',
 'Chico',
 'Chino',
 'Chino-Hills',
 'Chowchilla',
 'Chula-Vista',
 'Citrus-Heights',
 'Clar

In [365]:
len(list_of_cities)

459

In [366]:
clean_links = []
missing_cities = []

In [ ]:
clean_links = []
missing_cities = []

In [367]:
for city in list_of_cities:
    #only get links for cities not in clean_links
        backup_url = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(city)
        query  = "'https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html'".format(city)
        query = urllib.parse.quote_plus(query) # Format into URL encoding
        number_result = 3
        
        ua = UserAgent()
        
        google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
        t0 = time.time()
        response = requests.get(google_url, {"User-Agent": ua.random})
        response_delay = time.time() - t0
        time.sleep(10*response_delay)
        soup = BeautifulSoup(response.text, "html.parser")
        
        result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
        
        links = []
        for r in result_div:
            
            try:
                link = r.find('a', href = True)

                if link != '':
                    links.append(link['href'])
            except:
                continue
        flag = True
        for i, l in enumerate(links):
            clean = re.search('\/url\?q\=(.*)\&sa',l)
            if clean is None:
                continue
            if city in clean.group(1):
                clean_links.append(clean.group(1))
                flag = False
        if flag:
            missing_cities.append(city)
            clean_links.append(backup_url)

Not perfect so we will:
    *remove identical strings
    *remove strings that have the word county in them
    *track http error 404 if our self made url fails



In [372]:
#remove county results
without_counties = pd.Series(clean_links)
without_counties = without_counties.mask(pd.Series(clean_links).str.contains('County'))
without_counties = without_counties.dropna()

In [387]:
#delete duplicates
city_urls = without_counties.drop_duplicates()

In [430]:
economy_indexes = city_urls[city_urls.str.contains('Economy')].index

In [426]:
CCD_indexes = city_urls[city_urls.str.contains('CCD')].index

In [473]:
for i in economy_indexes:
    print(city_urls[i])

http://www.towncharts.com/California/Economy/Avalon-city-CA-Economy-data.html
http://www.towncharts.com/California/Economy/Blythe-city-CA-Economy-data.html
https://www.towncharts.com/California/Economy/Chowchilla-city-CA-Economy-data.html


In [474]:
for i in CCD_indexes:
    print(city_urls[i])

http://www.towncharts.com/California/Housing/Barstow-CCD-CA-Housing-data.html
http://www.towncharts.com/California/Housing/Bishop-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Carpinteria-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chico-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Coachella-Valley-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Central-Colusa-CCD-CA-Housing-data.html


In [434]:
indexes_to_replace = pd.Series.tolist(economy_indexes)+pd.Series.tolist(CCD_indexes)

In [436]:
indexes_to_replace

[22, 44, 79, 29, 41, 68, 74, 87, 92]

In [449]:
towns_to_fix = pd.Series(['Avalon','Blythe','Chowchilla','Barstow','Bishop','Carpinteria','Chico','Coachella','Colusa'],index = indexes_to_replace)

In [502]:
cop_city_urls = city_urls.copy()

In [514]:
for i,v in towns_to_fix.iteritems():
    cop_city_urls[i] = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(v)
    print(cop_city_urls[i])

https://www.towncharts.com/California/Housing/Avalon-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Blythe-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chowchilla-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Barstow-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Bishop-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Carpinteria-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chico-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Coachella-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Colusa-city-CA-Housing-data.html


In [526]:
len(cop_city_urls)

466

In [516]:
#delete duplicates if accidentally created any
searchurls = cop_city_urls.drop_duplicates()

In [517]:
len(searchurls)

461

A quick look through of the first 100 shows that Burbank has a duplicate with CDP and Bishop city has a duplicate or https vs http

In [518]:
print(searchurls[searchurls.str.contains('CDP')])

52    https://www.towncharts.com/California/Housing/...
dtype: object


In [519]:
searchurls[52]

'https://www.towncharts.com/California/Housing/Burbank-CDP-CA-Housing-data.html'

In [520]:
searchurls[searchurls.str.contains('Bishop')]

40    http://www.towncharts.com/California/Housing/B...
41    https://www.towncharts.com/California/Housing/...
dtype: object

In [521]:
searchurls = searchurls.drop(labels = [40,52])

In [522]:
searchurls = searchurls.reset_index(drop=True)

In [523]:
len(searchurls)

459

In [349]:
#without_counties.str.extract(r'Housing\/(.*)\-CA',expand=True)

,0
0,Agoura-Hills-city
1,Alameda-city
3,Albany-city
4,Alhambra-city
5,Aliso-Viejo-city
6,American-Canyon-city
7,Anaheim-city


In [534]:


# #to pickle
# outfile = open('search_urls','wb')
# pickle.dump(searchurls,outfile)
# outfile.close()

# outfile = open('my_clean_links','wb')
# pickle.dump(clean_links,outfile)
# outfile.close()

# outfile = open('unfound_urls','wb')
# pickle.dump(missing_cities,outfile)
# outfile.close()


In [406]:

#unpickling
# infile = open('search_urls','rb')
# search_urls = pickle.load(infile)
# infile.close()

# infile = open('my_clean_links','rb')
# clean_links = pickle.load(infile)
# infile.close()

# infile = open('unfound_urls','rb')
# unfound_urls = pickle.load(infile)
# infile.close()